# Problem statement

### Bike sharing assignment

### A US bike-sharing provider BoomBikes contracts a consulting company to understand the demands for their bike share business. 

#### Essentially they are interested in below
 - Significant vairables in predicting demand
 - How well those vairables describe the bike demand

 

# High level approach

- Understand and clean up the data
- EDA and inferences
- Data preparation
- Splitting data into training and test sets
- Build model 
- Residual analysis  
- Making predictions
- Evalute the model 
- Conclusion

## Understand the data

#### Importing all the libraries to perform the model building

In [2590]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# statsmodel libraries 
import statsmodels
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

# Sci kit learn libraries
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler

# Importing RFE and LinearRegression
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

In [2591]:
# loading the data
bike = pd.read_csv("day.csv")

In [2592]:
# Lets check the data 
bike.head()

,instant,dteday,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [2593]:
bike.describe()

,instant,season,yr,mnth,holiday,weekday,workingday,weathersit,temp,atemp,hum,windspeed,casual,registered,cnt
count,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000,730.000000
mean,365.500000,2.498630,0.500000,6.526027,0.028767,2.997260,0.683562,1.394521,20.319259,23.726322,62.765175,12.763620,849.249315,3658.757534,4508.006849
std,210.877136,1.110184,0.500343,3.450215,0.167266,2.006161,0.465405,0.544807,7.506729,8.150308,14.237589,5.195841,686.479875,1559.758728,1936.011647
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,2.424346,3.953480,0.000000,1.500244,2.000000,20.000000,22.000000
25%,183.250000,2.000000,0.000000,4.000000,0.000000,1.000000,0.000000,1.000000,13.811885,16.889713,52.000000,9.041650,316.250000,2502.250000,3169.750000
50%,365.500000,3.000000,0.500000,7.000000,0.000000,3.000000,1.000000,1.000000,20.465826,24.368225,62.625000,12.125325,717.000000,3664.500000,4548.500000
75%,547.750000,3.000000,1.000000,10.000000,0.000000,5.000000,1.000000,2.000000,26.880615,30.445775,72.989575,15.625589,1096.500000,4783.250000,5966.000000
max,730.000000,4.000000,1.000000,12.000000,1.000000,6.000000,1.000000,3.000000,35.328347,42.044800,97.250000,34.000021,3410.000000,6946.000000,8714.000000


In [2594]:
# checking for null values 
bike.isna().sum()

instant       0
dteday        0
season        0
yr            0
mnth          0
holiday       0
weekday       0
workingday    0
weathersit    0
temp          0
atemp         0
hum           0
windspeed     0
casual        0
registered    0
cnt           0
dtype: int64

#### No null values to handle 

In [2595]:
# renaming few columns for better readibility
bike.rename(columns={'yr':'year','mnth':'month','hum':'humidity'}, inplace=True)

In [2596]:
bike.head()

,instant,dteday,season,year,month,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,casual,registered,cnt
0,1,01-01-2018,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,331,654,985
1,2,02-01-2018,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,131,670,801
2,3,03-01-2018,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,120,1229,1349
3,4,04-01-2018,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,108,1454,1562
4,5,05-01-2018,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,82,1518,1600


In [2597]:
# Dropping instant as it is just the index of the record
# Dropping casual and registered as the target column 'cnt' has the summation already
# Dropping dteday as we have year and month already
bike.drop(['instant','casual','registered', 'dteday'],axis=1,inplace=True)

#### P.S. temp and atemp looks similiar but will check VIF and consider to keep them or not 

In [2598]:
bike.head()

,season,year,month,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,cnt
0,1,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985
1,1,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801
2,1,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349
3,1,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562
4,1,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600


In [2599]:
# Assigning categorical values from interpretred numerical values based on dat dictionary 
bike.season = bike.season.map({1:'spring', 2:'summer', 3:'fall', 4:'winter'})

In [2600]:
bike.head()

,season,year,month,holiday,weekday,workingday,weathersit,temp,atemp,humidity,windspeed,cnt
0,spring,0,1,0,6,0,2,14.110847,18.18125,80.5833,10.749882,985
1,spring,0,1,0,0,0,2,14.902598,17.68695,69.6087,16.652113,801
2,spring,0,1,0,1,1,1,8.050924,9.47025,43.7273,16.636703,1349
3,spring,0,1,0,2,1,1,8.200000,10.60610,59.0435,10.739832,1562
4,spring,0,1,0,3,1,1,9.305237,11.46350,43.6957,12.522300,1600


In [2601]:
bike.month = bike.month.map({1:'jan',2:'feb',3:'mar',4:'apr',5:'may',6:'june',7:'july',8:'aug',9:'sep',10:'oct',11:'nov',12:'dec'})

In [2602]:
# bike.weekday = bike.weekday.map({0:'sun',1:'mon',2:'tue',3:'wed',4:'thu',5:'fri',6:'sat'})

bike.weekday = bike.weekday.map({1:'Wed', 2:'Thurs', 3:'Fri', 4:'Sat', 5:'Sun', 6:'Mon', 7:'Tues'})

In [2603]:
bike.weathersit = bike.weathersit.map({1:'Clear',2:'Misty',3:'Light_Snow'})# need to check 
# bike.weathersit.unique()

In [2604]:
bike.shape

(730, 12)

In [2605]:
bike.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 730 entries, 0 to 729
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   season      730 non-null    object 
 1   year        730 non-null    int64  
 2   month       730 non-null    object 
 3   holiday     730 non-null    int64  
 4   weekday     625 non-null    object 
 5   workingday  730 non-null    int64  
 6   weathersit  730 non-null    object 
 7   temp        730 non-null    float64
 8   atemp       730 non-null    float64
 9   humidity    730 non-null    float64
 10  windspeed   730 non-null    float64
 11  cnt         730 non-null    int64  
dtypes: float64(4), int64(4), object(4)
memory usage: 68.6+ KB


##  EDA and inferences

In [ ]:
# pariplot numerical columns to understand correlation
numerical_columns = ["temp", "atemp", "humidity", "windspeed", "month", "cnt"]
sns.pairplot(bike[numerical_columns])
plt.show()

In [ ]:
plt.figure(figsize=(6,5),dpi=110)
plt.title("Cnt vs Temp",fontsize=16)
sns.regplot(data=bike,y="cnt",x="temp")
plt.xlabel("Temperature")
plt.show()

In [ ]:
plt.figure(figsize=(6,5),dpi=110)
plt.title("Cnt vs Humidity",fontsize=16)
sns.regplot(data=bike,y="cnt",x="humidity")
plt.xlabel("Humidity")
plt.show()

### Inferences 

- cnt has good correlation with temp hence this might have good impact on our model
- cnt has good correlation with atemp similar to temp
- temp and atemp pattern looks similar hence we should verify the multicollinearity during VIF checks 
- temp has slight negative correlation with windpseed 
- humidity has negative correlation with windspeed 

### Visualising categorical variables

In [ ]:
plt.figure(figsize=(20, 12))
plt.subplot(2,3,1)
sns.boxplot(x = 'weathersit', y='cnt', data = bike)
plt.subplot(2,3,2)
sns.boxplot(x = 'season', y='cnt', data = bike)
plt.subplot(2,3,3)
sns.boxplot(x = 'holiday', y='cnt', data = bike)
plt.subplot(2,3,4)
sns.boxplot(x = 'weekday', y='cnt', data = bike)
plt.subplot(2,3,5)
sns.boxplot(x = 'workingday', y='cnt', data = bike)
plt.subplot(2,3,6)
sns.boxplot(x = 'month', y='cnt', data = bike)
plt.show()

#### Inferences 
- Clearly the trend is high when there is clear weather (~ around 6k) with median of 5k 
- Fall and summer has high range of bike sharing with 75 percentile above 6k
- Saturday's and Monday seem to be high demand days
- Mid months of the year has high demand such as June, May which could correlate to the season 

### Multivariate and inferences 

In [ ]:
# season/month bi variate analysis
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
sns.barplot('month','cnt',data=bike)
plt.subplot(1,2,2)
sns.barplot('month','cnt',data=bike, hue='season',palette='Set1')
plt.show()

In [ ]:
# season/month bi variate analysis
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
sns.barplot('month','cnt',data=bike)
plt.subplot(1,2,2)
sns.barplot('month','cnt',data=bike, hue='holiday',palette='Set1')
plt.show()

In [ ]:
# year/Month uni variate and bi variate analysis
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
sns.barplot('month','cnt',data=bike)
plt.subplot(1,2,2)
sns.barplot('month','cnt',data=bike, hue='year',palette='Set1')
plt.show()

In [ ]:
plt.figure(figsize = (12,6))
plt.subplot(1,2,1)
sns.barplot('workingday','cnt',data=bike)
plt.subplot(1,2,2)
sns.barplot('workingday','cnt',data=bike, hue='year',palette='Set1')
plt.show()

In [ ]:
plt.figure(figsize=(6,5),dpi=110)
plt.title("Cnt vs windspeed",fontsize=16)
sns.scatterplot(hue="season",data=bike,y="cnt",x="windspeed")
plt.xlabel("humidity")
plt.show()

In [ ]:
plt.figure(figsize=(6,5),dpi=110)
plt.title("Cnt vs Temp",fontsize=16)
sns.scatterplot(hue="season",data=bike,y="cnt",x="temp")
plt.xlabel("Temperature")
plt.show()

### Inferences 
- Mid months clearly has high demand due to the season which proves our univariate analysis again
- Demand has increased in 2019 due to the post pandemic and all month in 2019 has higher deman than 2018 
- Working day has comparatively better demand than a holiday 

## Data Preparation

### Dumy variables 

In [ ]:
# Let's drop the first column from status df using 'drop_first = True' , combine with one command 
season_df = pd.get_dummies(bike['season'], drop_first = True)
weathersit_df = pd.get_dummies(bike['weathersit'], drop_first = True)
month_df = pd.get_dummies(bike['month'], drop_first = True)
weekday_df = pd.get_dummies(bike['weekday'], drop_first = True)

In [ ]:
# Add the results to the original day dataframe
bike = pd.concat([bike, season_df, month_df, weathersit_df, weekday_df], axis = 1)

In [ ]:
bike.info()

In [ ]:
# dropping unnecessary columns as we have already created dummy variable out of it.
bike.drop(['season','month','weekday','weathersit'], axis = 1, inplace = True)

In [ ]:
bike.info()

## Splitting the Data into Training and Testing Sets

In [ ]:
from sklearn.model_selection import train_test_split

# We specify this so that the train and test data set always have the same rows, respectively
np.random.seed(0)
bike_train, bike_test = train_test_split(bike, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
bike_train.shape

### Scaling the training data set

In [ ]:
scaler = MinMaxScaler()

In [ ]:
bike_train.head()

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['temp', 'atemp', 'humidity', 'windspeed', 'cnt']

# Fit and transform the training the day using scaler 
bike_train[num_vars] = scaler.fit_transform(bike_train[num_vars])

In [ ]:
# post scaling we can see max value is 1 for all coulumns
bike_train.describe()

## Building the model 

In [ ]:
# Building the Linear Model
y_train = bike_train.pop('cnt')
X_train = bike_train

In [ ]:
X_train.info()

### RFE 

In [ ]:
# Running RFE with the output number of the variable equal to 10
lm = LinearRegression()
lm.fit(X_train, y_train)

rfe = RFE(lm, n_features_to_select=15)             # running RFE
rfe = rfe.fit(X_train, y_train)

In [ ]:
# Verifying the response from RFE function for all features
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
# Extracting all columns picked by RFE
col = X_train.columns[rfe.support_]
col

In [ ]:
# Printing negated columns by RFE
X_train.columns[~rfe.support_]

### Building model using statsmodel, for the detailed statistics

In [ ]:
# Generic function to calculate VIF of variables

def calculateVIF(df):
    vif = pd.DataFrame()
    vif['Features'] = df.columns
    vif['VIF'] = [variance_inflation_factor(df.values, i) for i in range(df.shape[1])]
    vif['VIF'] = round(vif['VIF'], 2)
    vif = vif.sort_values(by = "VIF", ascending = False)
    return vif 

In [ ]:
# Creating X_test dataframe with RFE selected variables
X_train_rfe = X_train[col]

In [ ]:
X_train_rfe.info()

### Manual Elimination

In [ ]:
# Building model for second time after dropping humidity 
X_train_lm1 = sm.add_constant(X_train_rfe)
lr_1 = sm.OLS(y_train,X_train_lm1).fit()
print(lr_1.summary())

In [ ]:
#  checking VIF with RFE selected variables 
calculateVIF(X_train_rfe)

In [ ]:
# As humidity shows high VIF values hence we can drop it and taking new variable X_train_new
X_train_new = X_train_rfe.drop(['humidity'], axis = 1)

# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

In [ ]:
# Building model for second time after dropping humidity 
X_train_lm2 = sm.add_constant(X_train_new)
lr_2 = sm.OLS(y_train,X_train_lm2).fit()
print(lr_2.summary())

In [ ]:
# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

In [ ]:
# Dropping holiday as it has slighly high p value than others 
X_train_new.drop(['workingday'], axis = 1, inplace=True)

In [ ]:
# Building third model for second time after dropping workingday 
X_train_lm3 = sm.add_constant(X_train_new)
lr_3 = sm.OLS(y_train,X_train_lm3).fit()
print(lr_3.summary())

In [ ]:
# Dropping holiday as it has slighly high p value than others 
X_train_new.drop(['Mon'], axis = 1, inplace=True)

In [ ]:
# Building fourth model for second time after dropping workingday 
X_train_lm4 = sm.add_constant(X_train_new)
lr_4 = sm.OLS(y_train,X_train_lm4).fit()
print(lr_4.summary())

In [ ]:
# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

In [ ]:
# Dropping holiday as it has slighly high p value than others 
X_train_new.drop(['windspeed'], axis = 1, inplace=True)

In [ ]:
# Building fifth model for second time after dropping workingday 
X_train_lm5 = sm.add_constant(X_train_new)
lr_5 = sm.OLS(y_train,X_train_lm5).fit()
print(lr_5.summary())

In [ ]:
# Dropping holiday as it has slighly high p value than others 
X_train_new.drop(['jan'], axis = 1, inplace=True)

In [ ]:
# Building sixth model for second time after dropping workingday 
X_train_lm6 = sm.add_constant(X_train_new)
lr_6 = sm.OLS(y_train,X_train_lm6).fit()
print(lr_6.summary())

### So the model we predicted is given below

_cnt = 0.1503 + 0.5034 x temp + 0.2325 x year - 0.1008 x holiday - 0.0764 x spring + 0.0355 x summer 
       + 0.0842 x winter - 0.0527 x july + 0.0810 x sep - 0.2999 x Light_Snow - 0.0798 x Misty_ 


In [ ]:
X_train_lm6

In [ ]:
y_train_pred = lr_6.predict(X_train_lm6) 

##  Residual analysis

### Linearity 

In [ ]:
print("R squared: {}".format(r2_score(y_true=y_train,y_pred=y_train_pred)))

#### Training and Predicted R squared is 82% so we are good here

###  Check for Normality of error terms/residuals¶

In [ ]:
# Plot the histogram of the error terms
fig = plt.figure()
sns.distplot((y_train - y_train_pred), bins = 20)
fig.suptitle('Error Terms', fontsize = 20) 
plt.xlabel('Errors', fontsize = 18)
plt.show()

In [ ]:
### The scientific way for the mean of residuals 
residuals = y_train.values-y_train_pred
mean_residuals = np.mean(residuals)
print("Mean of Residuals {}".format(mean_residuals))

#### Close to zero hence all good here

### Multicollinearity

In [ ]:
# Run the function to calculate VIF for the new model
calculateVIF(X_train_new)

#### VIF value is lesser than the 5 for all the feature we chose hence we are good here

### Homoscedasticity

In [ ]:
p = sns.scatterplot(y_train_pred,residuals)
plt.xlabel('y_pred/predicted values')
plt.ylabel('Residuals')
plt.ylim(-1,2)
plt.xlim(0,1)
p = sns.lineplot([0,2],[0,0],color='blue')
p = plt.title('Residuals vs fitted values plot for homoscedasticity check')
plt.show()

#### The residuals have equal or almost equal variance across the regression line.

##  Making Predictions

In [ ]:
# Apply scaler() to all the columns except the 'yes-no' and 'dummy' variables
num_vars = ['temp', 'atemp', 'humidity', 'windspeed', 'cnt']

# Fit and transform the training the day using scaler 
bike_test[num_vars] = scaler.transform(bike_test[num_vars])

In [ ]:
bike_test.describe()

In [ ]:
# Lets create X and Y axis value from test set
y_test = bike_test.pop('cnt')
X_test = bike_test

In [ ]:
# Lets extract all the trained columns 
col1 = X_train_new.columns
X_test = X_test[col1]

# Adding constant variable to test dataframe
X_test_lm_6 = sm.add_constant(X_test)

In [ ]:
# Predicting the model for test set 
y_pred = lr_6.predict(X_test_lm_6)

In [ ]:
# print r square value of test data
r2 = r2_score(y_test, y_pred)
round(r2,5)

In [ ]:
# Print the adjusted_r2 for test set
adjusted_r2 = round(1-(1-r2)*(X_test.shape[0]-1)/(X_test.shape[0]-X_test.shape[1]-1),4)
print(adjusted_r2)

##  Model evaluation

In [ ]:
# Plotting y_test and y_pred to understand the spread.
fig = plt.figure()
plt.scatter(y_test,y_pred)
fig.suptitle('y_test vs y_pred', fontsize=20)              # Plot heading 
plt.xlabel('y_test', fontsize=18)                          # X-label
plt.ylabel('y_pred', fontsize=16)
plt.show()

## Final values and conclusion

- Train dataset R^2          : 0.824
- Test dataset R^2           : 0.8089
- Train dataset Adjusted R^2 : 0.821    
- Test dataset Adjusted R^2  : 0.7998

### Based on our model bike demand depends upon temp, year, spring, summer, winter, sep, holiday, july, Light_Snow and Misty